<h2>Problem Statement # 1 :  Getting ready to appear for a Trial</h2>

In [41]:
%%capture
!pip install PyPDF2
!pip install openai
!pip install nltk
!pip install python-dotenv

In [42]:
import platform
import os
import openai
import os
import re
import PyPDF2

In [43]:
#load api key from .env
from dotenv import load_dotenv
load_dotenv()
openai.api_key=os.getenv("OPENAI_API_KEY")

In [44]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

print('Python: ', platform.python_version())
print('re: ', re.__version__)
print('nltk: ', nltk.__version__)

Python:  3.9.0
re:  2.2.1
nltk:  3.8.1


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lnavn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
# Read the PDF document
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

In [46]:
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

In [47]:
text = read_pdf("ps1.pdf")
token_count = count_tokens(text)
print(f"Number of tokens: {token_count}")

Number of tokens: 7500


In [48]:
def break_up_file(tokens, chunk_size, overlap_size):
    if len(tokens) <= chunk_size:
        yield tokens
    else:
        chunk = tokens[:chunk_size]
        yield chunk
        yield from break_up_file(tokens[chunk_size-overlap_size:], chunk_size, overlap_size)

def break_up_file_to_chunks(text, chunk_size=2000, overlap_size=100):
    tokens = word_tokenize(text)
    return list(break_up_file(tokens, chunk_size, overlap_size))

In [49]:
chunks = break_up_file_to_chunks(text)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} tokens")

Chunk 0: 2000 tokens
Chunk 1: 2000 tokens
Chunk 2: 2000 tokens
Chunk 3: 1800 tokens


In [50]:
def convert_to_detokenized_text(tokenized_text):
    prompt_text = " ".join(tokenized_text)
    prompt_text = prompt_text.replace(" 's", "'s")
    return prompt_text

In [51]:
prompt_response = []
chunks = break_up_file_to_chunks(text)

In [53]:
for i, chunk in enumerate(chunks):
    prompt_request = "Please generate a concise summary for the following text with references to their corresponding page and line numbers:\n" + convert_to_detokenized_text(chunks[i])
    response = openai.Completion.create(
            model="text-davinci-003",
            prompt=prompt_request,
            temperature=.5,
            max_tokens=500,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
    )
    
    prompt_response.append(response["choices"][0]["text"].strip())

In [54]:
prompt_request = "Give the 10 most important items pertaining this trial along with their corresponding page and line numbers: " + str(prompt_response)

In [55]:
response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt_request,
        temperature=.5,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

In [56]:
import nltk
from nltk.tokenize import sent_tokenize

meeting_summary = response["choices"][0]["text"].strip()

# Split the meeting summary into sentences
sentences = sent_tokenize(meeting_summary)

# Print the sentences with bullet points
for i, sentence in enumerate(sentences, start=1):
    sentence = sentence.strip()
    if sentence != "":
        print(f"{sentence}")


1.
The FDA’s approval of mifepristone for mail-order distribution defies its own regulatory authority and longstanding federal criminal law, as well as the authority of states to protect the health, safety, and welfare of their citizens by regulating abortion.
(Page 15, Lines 1-17; Page 16, Lines 1-14; Page 17, Lines 1-14; Page 18, Lines 1-14; Page 19, Lines 1-2)

2.
The authority to regulate or prohibit abortion belongs to the citizens of each state and the FDA has no authority to make broad policy judgments balancing the people's interests in prenatal life, maternal health and safety, and the integrity of the medical profession (Dobbs, 142 S. Ct. at 2284).
(Page 19, Line 3)

3.
The FDA’s actions harm the public interest by undermining States’ ability to protect their citizens and by forcing States to divert scarce resources to investigating and prosecuting violations of their laws.
(Page 9, Lines 4-5)

4.
The FDA may approve drugs for interstate sale and distribution only after deter

In [57]:
#print the two-page summary
for i in prompt_response:
    print(i)

the authority to protect their citizens through abortion regulation . Third , the FDA ’ s actions harm the public interest by undermining States ’ ability to protect their citizens and by forcing States to divert scarce resources to investigating and prosecuting violations of their laws . Case 2:22-cv-00223-Z Document 100 Filed 02/13/23 Page 9 of 26 PageID 3768 4 BACKGROUND The FDA ’ s actions on mifepristone are governed by the Food , Drug , and Cosmetic Act ( FDCA ) , 21 U.S.C . § 301 et seq . Under the FDCA , the FDA may approve drugs for interstate sale and distribution only after determining that the drugs are “ safe and effective for use under the conditions prescribed , recommended , or suggested in the labeling thereof. ” 21 U.S.C . § 355(d) . The FDCA also requires that drugs be dispensed “ only by or upon a valid prescription of a practitioner licensed by law to administer such drug. ” 21 U.S.C . § 393 . The FDA ’ s authority to approve drugs is further circumscribed by the D